In [ ]:
import torch
import torchvision
from torchvision import transforms
from PIL import Image
from os import listdir
import random
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn

normalize = transforms.Normalize(
    mean = [0.485, 0.456, 0.406],
    std = [0.229, 0.224, 0.225]
)

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    normalize])

# TARGET: [isCat, isDog]
train_data_list = []
target_list = []
train_data = []
files = listdir('catdog/train/')
for i in range(len(listdir('catdog/train/'))) :
    f = random.choice(files)
    files.remove(f)
    img = Image.open("catdog/train/" + f )
    img_tensor = transform(img) #(3,256,256)
    train_data_list.append(img_tensor)

    isCat = 1 if 'cat' in f else 0
    isDog = 1 if 'dog' in f else 0
    target = [isCat,isDog]
    target_list.append(target)

    if len(train_data_list) >= 64:
       train_data.append((torch.stack(train_data_list),target_list))
       train_data_list = []
       target_list = []
       print('Loaded batch ', len(train_data), 'of', int(len(listdir('catdog/train/'))))
       print('Percentage Done: ', 100*len(train_data) /int(len(listdir('catdog/train/'))), 'of', int(len(listdir('catdog/train/'))))
       # print(train_data)
       if len(train_data) > 5: # change number of trainings
        break

class Netz(nn.Module):
    def __init__(self):
        super(Netz,self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 12, kernel_size=5)
        self.conv3 = nn.Conv2d(12, 18, kernel_size=5) #18*28*28 = 14'000 neurons
        self.conv4 = nn.Conv2d(18, 24, kernel_size=5)
        self.fc1 = nn.Linear(14112, 1000) # 1000 neurons
        self.fc2 = nn.Linear(1000, 2) # 2 neurons cat or dog our output

    def forward(self,x):
        x = self.conv1(x)
        x = F.max_pool2d(x,2)
        x = F.relu(x)

        x = self.conv2(x)
        x = F.max_pool2d(x,2)
        x = F.relu(x)

        x = self.conv3(x)
        x = F.max_pool2d(x,2)
        x = F.relu(x)
        x = x.view(-1, 14112) # nr of neurons
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.sigmoid(x) # neither softmax nor log_softmax dont use here use with binary_cross_entropy
    
        print(x.size())
        exit()

model = Netz()
model.to("mps") # no gpu on m1 mac, use mps

optimizer = optim.Adam(model.parameters(), lr=0.01)
def train(epoch) :
    model.train()
    batch_id = 0
    for data, target in train_data:
        data = data.to("mps")
        target = torch.Tensor(target).to("mps") #m1 without gpu .cuda(), use .to("mps") instead
        data = Variable(data)
        target = Variable(target)
        optimizer.zero_grad()
        out = model(data)
        criterion = F.binary_cross_entropy # not nll_loss
        loss = criterion(out,target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_id * len(data), len(train_data), 100 * batch_id / len(train_data), loss.data))
        batch_id = batch_id + 1

def test():
    model.eval();
    files = listdir('catdog/test/')
    f = random.choice(files)
    img = Image.open('catdog/test/' + f)
    img_eval_tensor = transform(img)
    img_eval_tensor.unsqueeze_(0)
    data = Variable(img_eval_tensor.to("mps"))
    out = model(data)
    isa = 'dog'
    if (int(out.data.max(1,keepdim=True)[1].tolist()[0][0]) == 0) :  # dim 1: 0 is Cat - 1 is dog
     isa = 'cat'
    
    print('is a: ' , isa)
    print(out.data.max(1,keepdim=True)[1])

    img.show()
    x = input('')

for epoch in range(1,30):
    train(epoch)
test()

Loaded batch  1 of 25000
Percentage Done:  0.004 of 25000
Loaded batch  2 of 25000
Percentage Done:  0.008 of 25000
Loaded batch  3 of 25000
Percentage Done:  0.012 of 25000
Loaded batch  4 of 25000
Percentage Done:  0.016 of 25000
Loaded batch  5 of 25000
Percentage Done:  0.02 of 25000
Loaded batch  6 of 25000
Percentage Done:  0.024 of 25000
Train Epoch: 1 [0/6 (0%)]	Loss: 0.693651
Train Epoch: 1 [64/6 (17%)]	Loss: 50.000000
Train Epoch: 1 [128/6 (33%)]	Loss: 57.812500
Train Epoch: 1 [192/6 (50%)]	Loss: 42.187500
Train Epoch: 1 [256/6 (67%)]	Loss: 40.625000
Train Epoch: 1 [320/6 (83%)]	Loss: 39.062500
Train Epoch: 2 [0/6 (0%)]	Loss: 46.875000
Train Epoch: 2 [64/6 (17%)]	Loss: 50.000000
Train Epoch: 2 [128/6 (33%)]	Loss: 57.812500
Train Epoch: 2 [192/6 (50%)]	Loss: 42.187500
Train Epoch: 2 [256/6 (67%)]	Loss: 40.625000
Train Epoch: 2 [320/6 (83%)]	Loss: 39.062500
Train Epoch: 3 [0/6 (0%)]	Loss: 46.875000
Train Epoch: 3 [64/6 (17%)]	Loss: 50.000000
Train Epoch: 3 [128/6 (33%)]	Loss: 5